In [77]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, DoubleType, IntegerType
from pyspark.sql.functions import col, struct, explode, when, lit, array_max, array, split, regexp_replace



In [78]:
# variant_srcdir = 's3://dig-analysis-data/out/varianteffect/common/part-*'
# outdir = 's3:/dig-analysis-data/out/varianteffect/magma/'

# development localhost directories
variant_srcdir = '/Users/mduby/Data/Broad/Magma/Common/part*'
out_dir = '/Users/mduby/Data/Broad/Magma/Out/Step1'

# print
print("the variant input directory is: {}".format(variant_srcdir))
print("the output directory is: {}".format(out_srcdir))



the variant input directory is: /Users/mduby/Data/Broad/Magma/Common/part*
the output directory is: /Users/mduby/Data/Broad/Magma/Out/Step1/


In [3]:
# open spark session
spark = SparkSession.builder.appName('bioindex').getOrCreate()

print("got Spark session of type {}".format(type(spark)))


got Spark session of type <class 'pyspark.sql.session.SparkSession'>


In [5]:
# this is the schema for the common variant file
variant_schema = StructType(
    [
        StructField('varId', StringType(), nullable=False),
        StructField('dbSNP', StringType(), nullable=False),
        StructField('consequence', StringType(), nullable=False),
        StructField('gene', StringType(), nullable=False),
        StructField('transcript', StringType(), nullable=False),
        StructField('impact', StringType(), nullable=False),
    ]
)


In [58]:
# method to load the frequencies
df_load = spark.read \
        .csv(variant_srcdir, sep='\t', header=True, schema=variant_schema) \
        .select('varId', 'dbSNP')# method to load the frequencies

# print
print("the loaded variant data frame has {} rows".format(df_load.count()))
df_load.show()
        

the loaded variant data frame has 67003328 rows
+---------------+-----------+
|          varId|      dbSNP|
+---------------+-----------+
|1:62185338:AT:A|       null|
| 1:62190015:G:T|rs147606427|
| 1:62190786:C:T|       null|
| 1:62192716:A:G|rs185779444|
| 1:62197508:T:C|rs114282349|
| 1:62204697:C:G|rs554118633|
| 1:62204701:G:T|rs575749077|
|  1:6220708:C:T|rs546938756|
| 1:62208536:G:C|rs183841374|
| 1:62212635:C:A|rs184384859|
|  1:6221516:C:T|rs902947089|
| 1:62228647:G:A|rs368696281|
| 1:62234945:C:T|       null|
| 1:62236787:C:A|rs748365709|
| 1:62250649:G:T| rs11207836|
| 1:62259655:A:T| rs12116549|
| 1:62267300:G:A|       null|
| 1:62269149:A:G|rs549300954|
| 1:62270403:G:C| rs74568317|
| 1:62272329:C:A|       null|
+---------------+-----------+
only showing top 20 rows



In [59]:
# keep only the rows with non null dbSNP ids
df_nonnull_load = df_load.filter(col("dbSNP").isNotNull())

# print
print("the non null RS id dataframe has {} rows".format(df_nonnull_load.count()))

the non null RS id dataframe has 59203797 rows


In [60]:
df_nonnull_load.show()

+--------------+-----------+
|         varId|      dbSNP|
+--------------+-----------+
|1:62190015:G:T|rs147606427|
|1:62192716:A:G|rs185779444|
|1:62197508:T:C|rs114282349|
|1:62204697:C:G|rs554118633|
|1:62204701:G:T|rs575749077|
| 1:6220708:C:T|rs546938756|
|1:62208536:G:C|rs183841374|
|1:62212635:C:A|rs184384859|
| 1:6221516:C:T|rs902947089|
|1:62228647:G:A|rs368696281|
|1:62236787:C:A|rs748365709|
|1:62250649:G:T| rs11207836|
|1:62259655:A:T| rs12116549|
|1:62269149:A:G|rs549300954|
|1:62270403:G:C| rs74568317|
|1:62274741:G:A|rs116618819|
|1:62275301:C:T|rs111792290|
|1:62279885:G:A|  rs2492282|
|1:62287883:A:G|rs781701525|
|1:62301139:G:A|rs549410823|
+--------------+-----------+
only showing top 20 rows



In [61]:
# decompose first field and get chrom/pos
split_col = split(df_nonnull_load['varId'], ':')

# add the first two columns back in
df_nonnull_load = df_nonnull_load.withColumn('chromosome', split_col.getItem(0))
df_nonnull_load = df_nonnull_load.withColumn('position', split_col.getItem(1))



In [62]:
df_nonnull_load.show()

+--------------+-----------+----------+--------+
|         varId|      dbSNP|chromosome|position|
+--------------+-----------+----------+--------+
|1:62190015:G:T|rs147606427|         1|62190015|
|1:62192716:A:G|rs185779444|         1|62192716|
|1:62197508:T:C|rs114282349|         1|62197508|
|1:62204697:C:G|rs554118633|         1|62204697|
|1:62204701:G:T|rs575749077|         1|62204701|
| 1:6220708:C:T|rs546938756|         1| 6220708|
|1:62208536:G:C|rs183841374|         1|62208536|
|1:62212635:C:A|rs184384859|         1|62212635|
| 1:6221516:C:T|rs902947089|         1| 6221516|
|1:62228647:G:A|rs368696281|         1|62228647|
|1:62236787:C:A|rs748365709|         1|62236787|
|1:62250649:G:T| rs11207836|         1|62250649|
|1:62259655:A:T| rs12116549|         1|62259655|
|1:62269149:A:G|rs549300954|         1|62269149|
|1:62270403:G:C| rs74568317|         1|62270403|
|1:62274741:G:A|rs116618819|         1|62274741|
|1:62275301:C:T|rs111792290|         1|62275301|
|1:62279885:G:A|  rs

In [63]:
# build out data frame and save magma variant input file
df_export = df_nonnull_load.select("dbSnp", 'chromosome', 'position')



In [64]:
df_export.count()

59203797

In [65]:
# replace the X/Y chromosome values with 23/24
df_export = df_export.withColumn('chromosome', regexp_replace('chromosome', 'X', '23'))
df_export = df_export.withColumn('chromosome', regexp_replace('chromosome', 'Y', '24'))



In [66]:
df_export.count()

59203797

In [55]:
df_export.printSchema()


root
 |-- dbSnp: string (nullable = true)
 |-- chromosome: string (nullable = true)
 |-- position: string (nullable = true)



In [80]:
# show the counts
df_export.groupBy("chromosome").count().orderBy("chromosome").show(25, False)


+----------+-------+
|chromosome|count  |
+----------+-------+
|1         |4650747|
|10        |2758809|
|11        |2877054|
|12        |2756916|
|13        |1952956|
|14        |1862917|
|15        |1707559|
|16        |1898522|
|17        |1843501|
|18        |1551438|
|19        |1483010|
|2         |4881079|
|20        |1302927|
|21        |762250 |
|22        |818314 |
|23        |2099694|
|24        |100    |
|3         |4041006|
|4         |3893393|
|5         |3658558|
|6         |3523628|
|7         |3278655|
|8         |3113723|
|9         |2486844|
+----------+-------+



In [69]:
df_export = df_export.filter(col("chromosome") != 'MT')

In [79]:
# show the counts
df_export.groupBy("chromosome").count().orderBy("chromosome").show(25, False)

+----------+-------+
|chromosome|count  |
+----------+-------+
|1         |4650747|
|10        |2758809|
|11        |2877054|
|12        |2756916|
|13        |1952956|
|14        |1862917|
|15        |1707559|
|16        |1898522|
|17        |1843501|
|18        |1551438|
|19        |1483010|
|2         |4881079|
|20        |1302927|
|21        |762250 |
|22        |818314 |
|23        |2099694|
|24        |100    |
|3         |4041006|
|4         |3893393|
|5         |3658558|
|6         |3523628|
|7         |3278655|
|8         |3113723|
|9         |2486844|
+----------+-------+



In [85]:
# write by chromosome
for chrom in range(1, 3):
    df_write = df_export.filter(col('chromosome') == chrom)
    # write out the tab delimited file
    print("chrom {} has size {}".format(chrom, df_write.count()))
    df_write.write.mode('overwrite').option("delimiter", "\t").csv(out_dir + "/" + str(chrom))

chrom 1 has size 4650747
chrom 2 has size 4881079


In [83]:
# write out by chrom
df_export.write.mode('overwrite').option("delimiter", "\t").partitionBy("chromosome").saveAsTable(out_dir)

ParseException: "\nextraneous input '/' expecting {'SELECT', 'FROM', 'ADD', 'AS', 'ALL', 'ANY', 'DISTINCT', 'WHERE', 'GROUP', 'BY', 'GROUPING', 'SETS', 'CUBE', 'ROLLUP', 'ORDER', 'HAVING', 'LIMIT', 'AT', 'OR', 'AND', 'IN', NOT, 'NO', 'EXISTS', 'BETWEEN', 'LIKE', RLIKE, 'IS', 'NULL', 'TRUE', 'FALSE', 'NULLS', 'ASC', 'DESC', 'FOR', 'INTERVAL', 'CASE', 'WHEN', 'THEN', 'ELSE', 'END', 'JOIN', 'CROSS', 'OUTER', 'INNER', 'LEFT', 'SEMI', 'RIGHT', 'FULL', 'NATURAL', 'ON', 'PIVOT', 'LATERAL', 'WINDOW', 'OVER', 'PARTITION', 'RANGE', 'ROWS', 'UNBOUNDED', 'PRECEDING', 'FOLLOWING', 'CURRENT', 'FIRST', 'AFTER', 'LAST', 'ROW', 'WITH', 'VALUES', 'CREATE', 'TABLE', 'DIRECTORY', 'VIEW', 'REPLACE', 'INSERT', 'DELETE', 'INTO', 'DESCRIBE', 'EXPLAIN', 'FORMAT', 'LOGICAL', 'CODEGEN', 'COST', 'CAST', 'SHOW', 'TABLES', 'COLUMNS', 'COLUMN', 'USE', 'PARTITIONS', 'FUNCTIONS', 'DROP', 'UNION', 'EXCEPT', 'MINUS', 'INTERSECT', 'TO', 'TABLESAMPLE', 'STRATIFY', 'ALTER', 'RENAME', 'ARRAY', 'MAP', 'STRUCT', 'COMMENT', 'S

In [71]:
# write out the tab delimited file
df_export.write.mode('overwrite').option("delimiter", "\t").csv(out_dir)

In [ ]:

# example

#    by_phenotype.drop(['rank', 'top']) \
#         .orderBy(['phenotype', 'pValue']) \
#         .write \
#         .mode('overwrite') \
#         .json('%s/phenotype' % outdir)

